In [1]:
from sympy import *

to use variables instead of symbols, the var keyword is helpful 

In [2]:
x,y,z = var('x y z')

In [14]:
def val_f():
    return z**y

In [15]:
val_f()

z**y

In [16]:
z = 2

In [17]:
val_f()

2**y

pi and cos are built in to sympy

In [18]:
pi

pi

In [19]:
cos(pi)

-1

common symbols are defined in sympy.abc

In [23]:
import sympy.abc
dir(sympy.abc)  

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_clash',
 '_clash1',
 '_clash2',
 'a',
 'alpha',
 'b',
 'beta',
 'c',
 'chi',
 'd',
 'delta',
 'division',
 'e',
 'epsilon',
 'eta',
 'exec_',
 'f',
 'g',
 'gamma',
 'greeks',
 'h',
 'i',
 'iota',
 'j',
 'k',
 'kappa',
 'l',
 'lamda',
 'm',
 'mu',
 'n',
 'nu',
 'o',
 'omega',
 'omicron',
 'p',
 'phi',
 'pi',
 'print_function',
 'psi',
 'q',
 'r',
 'rho',
 's',
 'sigma',
 'string',
 't',
 'tau',
 'theta',
 'u',
 'upsilon',
 'v',
 'w',
 'x',
 'xi',
 'y',
 'z',
 'zeta']

python floats have numerical precision issues

In [24]:
6.2

6.2

In [25]:
S(6.2)

6.20000000000000

to handle fractional exponents, use S(var) on one of the ints

In [39]:
u=y**(1/2)

In [40]:
u

1

In [41]:
u = y**(S(1)/S(2))

In [42]:
u

sqrt(y)

In [44]:
y

y

To solve an expression, use the solve command 

In [46]:
solve(x**2 - x,x)

[0]

but x was a variable, to reset this redefine x

In [47]:
x,y,z = symbols('x y z')

In [50]:
solve(2*x - 1,x)

[1/2]

or import division

In [51]:
from __future__ import division

In [52]:
x = y**(1/2)

In [53]:
x

y**0.5

to use varying precision 

In [55]:
Float(1.383843, '20')

1.3838429999999999342

In [56]:
S(1.383843, '20')

1.38384300000000

to make a substitution 

In [59]:
x,y,z = symbols('x y z')

In [60]:
u = x**2 + 2*y + z

In [61]:
u

x**2 + 2*y + z

In [62]:
u.subs({x:a,y:b})

NameError: name 'a' is not defined

In [63]:
a,b = symbols('a b')

In [64]:
u.subs({x:a,y:b})

a**2 + 2*b + z

In [65]:
u.subs({a:2,b:3})

x**2 + 2*y + z

In [66]:
u.subs({x:3,y:2})

z + 13

to convert a string to a sympy expression

In [87]:
str_expr = "x**2 + 3*x - 1/2"
expr = sympify(str_expr)
expr


x**2 + 3*x - 1/2

to walk the AST in sympy 

In [88]:
srepr(expr)

"Add(Pow(Symbol('x'), Integer(2)), Mul(Integer(3), Symbol('x')), Rational(-1, 2))"

In [89]:
Pow(2,x)

2**x

In [90]:
expr.args

(-1/2, x**2, 3*x)

In [91]:
expr.func(*expr.args)

x**2 + 3*x - 1/2

In [92]:
expr.args[2]

3*x

In [93]:
expr.args[2].args

(3, x)

In [74]:
pre(expr)

NameError: name 'pre' is not defined

In [76]:
def pre(expr):
    print(expr)
    for arg in expr.args:
        pre(arg)

In [94]:
pre(expr)

x**2 + 3*x - 1/2
-1/2
x**2
x
2
3*x
3
x


In [95]:
for arg in preorder_traversal(expr):
    print(arg)

x**2 + 3*x - 1/2
-1/2
x**2
x
2
3*x
3
x


In [96]:
expr.args[2].args

(3, x)

expression modification - expressions are immutable, so they cannot be modified, however new expressions can be created.

In [110]:
expr.args[0] + expr.args[1] + expr.args[2].subs(expr.args[2].args[0],y)

x**2 + x*y - 1/2

In [ ]:
an example of a hack code that recursively appends y when x is found

In [227]:
expr4 = []
def edit(expr,expr3):
    
    for arg in expr.args:
        print arg
        if arg == x:
            expr3.append(y)
            #edit(y,expr3)
        else:
            if arg.args:
                edit(arg,expr3)
                #expr3.append(arg)
            else:
                expr3.append(arg)

In [226]:
edit(expr,expr4)

-1/2


AttributeError: 'tuple' object has no attribute 'append'

In [220]:
expr4

[-1/2, y, 2, 3, y]

In [221]:
srepr(expr4)

"[Rational(-1, 2), Symbol('y'), Integer(2), Integer(3), Symbol('y')]"

expr4.func

In [224]:
expr.func(*expr4)

2*y + 9/2

gaining some information on sympy expression modification from this GSOC blog
https://jcrist.github.io/gsoc-week-7.html

In [228]:
def crawl(expr, func, *args, **kwargs):
    """Crawl the expression tree, and apply func to every node."""
    val = func(expr, *args, **kwargs)
    if val is not None:
        return val
    new_args = (crawl(arg, func, *args, **kwargs) for arg in expr.args)
    return expr.func(*new_args)

def printer(expr):
    """Print out every node"""
    print(expr)
    if not expr.args:
        return expr

print_expr = lambda expr: crawl(expr, printer)

In [258]:
print_expr( expr)

x**2 + 3*x - 1/2
-1/2
x**2
x
2
3*x
3
x


x**2 + 3*x - 1/2

In [265]:
def modify_funcs(expr):

    print expr.func

    if not expr.args:
        return expr   

In [266]:
mod_expr = lambda expr: crawl(expr, modify_funcs)
mod_expr(expr)

<class 'sympy.core.add.Add'>
<class 'sympy.core.numbers.Rational'>
<class 'sympy.core.power.Pow'>
<class 'sympy.core.symbol.Symbol'>
<class 'sympy.core.numbers.Integer'>
<class 'sympy.core.mul.Mul'>
<class 'sympy.core.numbers.Integer'>
<class 'sympy.core.symbol.Symbol'>


x**2 + 3*x - 1/2

In [236]:
expr.args[1].func 

sympy.core.power.Pow

to modify these maybe a tree structure is needed
so to save time i consult stack overflow
http://stackoverflow.com/questions/2358045/how-can-i-implement-a-tree-in-python-are-there-any-built-in-data-structures-in

In [269]:
import uuid

def sanitize_id(id):
    return id.strip().replace(" ", "")

(_ADD, _DELETE, _INSERT) = range(3)
(_ROOT, _DEPTH, _WIDTH) = range(3)

class Node:

    def __init__(self, name, identifier=None, expanded=True):
        self.__identifier = (str(uuid.uuid1()) if identifier is None else
                sanitize_id(str(identifier)))
        self.name = name
        self.expanded = expanded
        self.__bpointer = None
        self.__fpointer = []

    @property
    def identifier(self):
        return self.__identifier

    @property
    def bpointer(self):
        return self.__bpointer

    @bpointer.setter
    def bpointer(self, value):
        if value is not None:
            self.__bpointer = sanitize_id(value)

    @property
    def fpointer(self):
        return self.__fpointer

    def update_fpointer(self, identifier, mode=_ADD):
        if mode is _ADD:
            self.__fpointer.append(sanitize_id(identifier))
        elif mode is _DELETE:
            self.__fpointer.remove(sanitize_id(identifier))
        elif mode is _INSERT:
            self.__fpointer = [sanitize_id(identifier)]

class Tree:

    def __init__(self):
        self.nodes = []

    def get_index(self, position):
        for index, node in enumerate(self.nodes):
            if node.identifier == position:
                break
        return index

    def create_node(self, name, identifier=None, parent=None):

        node = Node(name, identifier)
        self.nodes.append(node)
        self.__update_fpointer(parent, node.identifier, _ADD)
        node.bpointer = parent
        return node

    def show(self, position, level=_ROOT):
        queue = self[position].fpointer
        if level == _ROOT:
            print "{0} [{1}]".format(self[position].name,
                                     self[position].identifier)
        else:
            print "\t"*level, "{0} [{1}]".format(self[position].name,
                                                 self[position].identifier)
        if self[position].expanded:
            level += 1
            for element in queue:
                self.show(element, level)  # recursive call

    def expand_tree(self, position, mode=_DEPTH):
        # Python generator. Loosly based on an algorithm from 'Essential LISP' by
        # John R. Anderson, Albert T. Corbett, and Brian J. Reiser, page 239-241
        yield position
        queue = self[position].fpointer
        while queue:
            yield queue[0]
            expansion = self[queue[0]].fpointer
            if mode is _DEPTH:
                queue = expansion + queue[1:]  # depth-first
            elif mode is _WIDTH:
                queue = queue[1:] + expansion  # width-first

    def is_branch(self, position):
        return self[position].fpointer

    def __update_fpointer(self, position, identifier, mode):
        if position is None:
            return
        else:
            self[position].update_fpointer(identifier, mode)

    def __update_bpointer(self, position, identifier):
        self[position].bpointer = identifier

    def __getitem__(self, key):
        return self.nodes[self.get_index(key)]

    def __setitem__(self, key, item):
        self.nodes[self.get_index(key)] = item

    def __len__(self):
        return len(self.nodes)

    def __contains__(self, identifier):
        return [node.identifier for node in self.nodes
                if node.identifier is identifier]

if __name__ == "__main__":

    tree = Tree()
    tree.create_node("Harry", "harry")  # root node
    tree.create_node("Jane", "jane", parent = "harry")
    tree.create_node("Bill", "bill", parent = "harry")
    tree.create_node("Joe", "joe", parent = "jane")
    tree.create_node("Diane", "diane", parent = "jane")
    tree.create_node("George", "george", parent = "diane")
    tree.create_node("Mary", "mary", parent = "diane")
    tree.create_node("Jill", "jill", parent = "george")
    tree.create_node("Carol", "carol", parent = "jill")
    tree.create_node("Grace", "grace", parent = "bill")
    tree.create_node("Mark", "mark", parent = "jane")

    print "="*80 
    tree.show( "harry")
    print "="*80
    for node in tree.expand_tree("harry", mode=_WIDTH):
        print node
    print "="*80

Harry [harry]
	Jane [jane]
		Joe [joe]
		Diane [diane]
			George [george]
				Jill [jill]
					Carol [carol]
			Mary [mary]
		Mark [mark]
	Bill [bill]
		Grace [grace]
harry
jane
bill
joe
diane
mark
grace
george
mary
jill
carol


In [270]:
def create_test_tree():
    tree = Tree()
    tree.create_node("Harry", "harry")  # root node
    tree.create_node("Jane", "jane", parent = "harry")
    tree.create_node("Harry", "bill", parent = "harry")
    tree.create_node("Joe", "joe", parent = "jane")
    tree.create_node("Diane", "diane", parent = "jane")
    tree.create_node("George", "george", parent = "diane")
    tree.create_node("Mary", "mary", parent = "diane")
    tree.create_node("Jill", "jill", parent = "george")
    tree.create_node("Carol", "carol", parent = "jill")
    tree.create_node("Grace", "grace", parent = "bill")
    tree.create_node("Mark", "mark", parent = "jane")

    print "="*80 
    tree.show( "harry")
    print "="*80
    for node in tree.expand_tree("harry", mode=_WIDTH):
        print node
    print "="*80

for this to be useful, the node identifier (capital letter names) must be repeatable, for these will contain either terminal nodes or operator nodes

In [271]:
defcreate_test_tree()

Harry [harry]
	Jane [jane]
		Joe [joe]
		Diane [diane]
			George [george]
				Jill [jill]
					Carol [carol]
			Mary [mary]
		Mark [mark]
	Harry [bill]
		Grace [grace]
harry
jane
bill
joe
diane
mark
grace
george
mary
jill
carol


In [282]:
def add_nodes(expr):
    print expr
    if not expr.args:
        print "{{}}"
        return expr
    #else:
        

add_nodes_lambda = lambda expr: crawl(expr, add_nodes)
add_nodes_lambda(expr)
        


x**2 + 3*x - 1/2
-1/2
{{}}
x**2
x
{{}}
2
{{}}
3*x
3
{{}}
x
{{}}


In [329]:
def add_nodes2(expr):
    print expr
    if not expr.args:
        #print expr.func
        return expr
    else:
        print expr.func
    #else:
        

add_nodes2_lambda = lambda expr: crawl(expr, add_nodes2)
test_add_nodes2 = add_nodes2_lambda(expr)

x**2 + 3*x - 1/2
<class 'sympy.core.add.Add'>
-1/2
x**2
<class 'sympy.core.power.Pow'>
x
2
3*x
<class 'sympy.core.mul.Mul'>
3
x


In [330]:
test_add_nodes2

x**2 + 3*x - 1/2

In [332]:
def add_nodes3(expr):
    
    if not expr.args:
        #print expr.func
        print expr
        return expr
    else:
        print expr.func
    #else:
        

add_nodes3_lambda = lambda expr: crawl(expr, add_nodes3)
test_add_nodes3 = add_nodes3_lambda(expr)

<class 'sympy.core.add.Add'>
-1/2
<class 'sympy.core.power.Pow'>
x
2
<class 'sympy.core.mul.Mul'>
3
x


In [333]:
test_add_nodes3

x**2 + 3*x - 1/2

In [289]:
expr.func

sympy.core.add.Add

In [291]:
expr.func == Add

True

In [335]:
def add_nodes3(expr):
    
    if not expr.args:
        #print expr.func
        print expr
        return expr
    else:
        if expr.func == Add:
            print Mul
        else:
            print expr.func
    #else:
        

add_nodes3_lambda = lambda expr: crawl(expr, add_nodes3)
test_add_nodes3 = add_nodes3_lambda(expr)

<class 'sympy.core.mul.Mul'>
-1/2
<class 'sympy.core.power.Pow'>
x
2
<class 'sympy.core.mul.Mul'>
3
x


what is a "*kwarg" anyway ? (from the function "crawl") ... the problem here is that we need crawl to assign new funcs instead of just calling a user defined function that will return optional expression args

In [324]:
new_expr3
def add_nodes4(expr, *new_expr):
    
    if not expr.args:
        #print expr.func
        print expr
        return expr
    else:
        new_expr = expr.func(*expr.args)
        print expr.func
    #else:



add_nodes4_lambda = lambda expr, *new_expr3: crawl(expr, add_nodes4, *new_expr3)
add_nodes4_lambda(expr, new_expr3)
new_expr3

<class 'sympy.core.add.Add'>
-1/2
<class 'sympy.core.power.Pow'>
x
2
<class 'sympy.core.mul.Mul'>
3
x


0

In [356]:
def print_expr1(expr):
   
    print expr.func
    for arg in expr.args:
        if not arg.args:
            print arg
        else:
            print_expr1(arg)

print_expr1(expr)

<class 'sympy.core.add.Add'>
-1/2
<class 'sympy.core.power.Pow'>
x
2
<class 'sympy.core.mul.Mul'>
3
x


In [357]:
c = x/y
c.func
print_expr1(c)

<class 'sympy.core.mul.Mul'>
x
<class 'sympy.core.power.Pow'>
y
-1


ok we now have enough to start developing a basic Genetic programming algorithm
Requirements
1) Mutation Operator
2) Crossover Operator

The Crossover operator is one of the more difficult problems because this involves swapping an entire
subtree. So starting with a basic mutation operation, we can swap out Add nodes with Mul nodes ...


In [359]:
def print_expr(expr):
   
    if expr.func == Add:
        expr_func = Mul
    else:
        expr_func = expr.func
        
    print expr_func
    for arg in expr.args:
        if not arg.args:
            print arg
        else:
            print_expr(arg)

    return expr_func(*expr.args)

new_test = print_expr(expr)

#print the new modified expression
new_test

<class 'sympy.core.mul.Mul'>
-1/2
<class 'sympy.core.power.Pow'>
x
2
<class 'sympy.core.mul.Mul'>
3
x


-3*x**3/2

In [360]:
#so ... the next task is to import random numbers ...
import random
[ random.randint(1, 6) for _ in range(10) ]



[1, 5, 4, 6, 1, 6, 2, 4, 4, 1]

In [ ]:


def create_test_tree(expr):
    tree = Tree()
    tree.create_node("op", expr.func)
    #for arg
    tree.create_node("Harry", "harry")  # root node
    tree.create_node("Jane", "jane", parent = "harry")
    tree.create_node("Harry", "bill", parent = "harry")
    tree.create_node("Joe", "joe", parent = "jane")
    tree.create_node("Diane", "diane", parent = "jane")
    tree.create_node("George", "george", parent = "diane")
    tree.create_node("Mary", "mary", parent = "diane")
    tree.create_node("Jill", "jill", parent = "george")
    tree.create_node("Carol", "carol", parent = "jill")
    tree.create_node("Grace", "grace", parent = "bill")
    tree.create_node("Mark", "mark", parent = "jane")

    print "="*80 
    tree.show( "harry")
    print "="*80
    for node in tree.expand_tree("harry", mode=_WIDTH):
        print node
    print "="*80

unevaluated expressions 

In [82]:

expr = Add(x, x, evaluate=False)

In [83]:
expr

x + x

In [84]:
sympify("x + x", evaluate=False)

x + x

In [85]:
x+expr

3*x